# Evolver Loop 9 Analysis

Verify exp_009 solution and understand the improvement.

In [ ]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely.geometry import Polygon
from shapely import affinity
from shapely.ops import unary_union
import json

getcontext().prec = 30
SCALE_FACTOR = Decimal('1e18')

print('Setup complete')

In [ ]:
# Tree shape vertices
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

def create_tree_polygon(x, y, angle):
    x, y, angle = float(x), float(y), float(angle)
    coords = list(zip(TX, TY))
    poly = Polygon(coords)
    poly = affinity.rotate(poly, angle, origin=(0, 0))
    poly = affinity.translate(poly, x, y)
    return poly

def create_high_precision_tree(x, y, angle):
    x = Decimal(str(x))
    y = Decimal(str(y))
    angle = Decimal(str(angle))
    sf = SCALE_FACTOR
    vertices = [
        (float(Decimal('0.0') * sf), float(Decimal('0.8') * sf)),
        (float(Decimal('0.125') * sf), float(Decimal('0.5') * sf)),
        (float(Decimal('0.0625') * sf), float(Decimal('0.5') * sf)),
        (float(Decimal('0.2') * sf), float(Decimal('0.25') * sf)),
        (float(Decimal('0.1') * sf), float(Decimal('0.25') * sf)),
        (float(Decimal('0.35') * sf), float(Decimal('0.0') * sf)),
        (float(Decimal('0.075') * sf), float(Decimal('0.0') * sf)),
        (float(Decimal('0.075') * sf), float(Decimal('-0.2') * sf)),
        (float(Decimal('-0.075') * sf), float(Decimal('-0.2') * sf)),
        (float(Decimal('-0.075') * sf), float(Decimal('0.0') * sf)),
        (float(Decimal('-0.35') * sf), float(Decimal('0.0') * sf)),
        (float(Decimal('-0.1') * sf), float(Decimal('0.25') * sf)),
        (float(Decimal('-0.2') * sf), float(Decimal('0.25') * sf)),
        (float(Decimal('-0.0625') * sf), float(Decimal('0.5') * sf)),
        (float(Decimal('-0.125') * sf), float(Decimal('0.5') * sf)),
    ]
    poly = Polygon(vertices)
    poly = affinity.rotate(poly, float(angle), origin=(0, 0))
    poly = affinity.translate(poly, xoff=float(x * sf), yoff=float(y * sf))
    return poly

def validate_no_overlap_strict(trees_data):
    if len(trees_data) <= 1:
        return True
    polygons = [create_high_precision_tree(t['x'], t['y'], t['deg']) for t in trees_data]
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            if polygons[i].intersects(polygons[j]) and not polygons[i].touches(polygons[j]):
                return False
    return True

def get_bbox_side(trees):
    if len(trees) == 0:
        return 0
    polygons = [create_tree_polygon(t['x'], t['y'], t['deg']) for t in trees]
    union = unary_union(polygons)
    bounds = union.bounds
    return max(bounds[2] - bounds[0], bounds[3] - bounds[1])

def get_score(trees, n):
    side = get_bbox_side(trees)
    return (side ** 2) / n

print('Functions defined')

In [ ]:
# Load exp_009 submission
print('Loading exp_009 submission...')
df = pd.read_csv('/home/code/experiments/009_santa2025_ensemble/submission.csv')
df['N'] = df['id'].astype(str).str.split('_').str[0].astype(int)

trees_009 = {}
scores_009 = {}

for n, g in df.groupby('N'):
    trees = []
    for _, row in g.iterrows():
        x = str(row['x']).replace('s', '')
        y = str(row['y']).replace('s', '')
        deg = str(row['deg']).replace('s', '')
        trees.append({'x': x, 'y': y, 'deg': deg})
    trees_009[n] = trees
    scores_009[n] = get_score(trees, n)

total_009 = sum(scores_009.values())
print(f'exp_009 score: {total_009:.6f}')
print(f'Target: 68.888293')
print(f'Gap: {total_009 - 68.888293:.6f}')

In [ ]:
# Validate all N values
print('\nValidating all N values...')
overlaps = []
for n in range(1, 201):
    if not validate_no_overlap_strict(trees_009[n]):
        overlaps.append(n)
    if n % 50 == 0:
        print(f'  Validated N=1-{n}')

if overlaps:
    print(f'\nWARNING: {len(overlaps)} N values have overlaps: {overlaps[:10]}...')
else:
    print('\n✅ All 200 N values pass strict validation!')

In [ ]:
# Compare with exp_008
print('\nComparing with exp_008...')
df_008 = pd.read_csv('/home/code/experiments/008_fractional_translation/submission.csv')
df_008['N'] = df_008['id'].astype(str).str.split('_').str[0].astype(int)

scores_008 = {}
for n, g in df_008.groupby('N'):
    trees = []
    for _, row in g.iterrows():
        x = str(row['x']).replace('s', '')
        y = str(row['y']).replace('s', '')
        deg = str(row['deg']).replace('s', '')
        trees.append({'x': x, 'y': y, 'deg': deg})
    scores_008[n] = get_score(trees, n)

total_008 = sum(scores_008.values())
print(f'exp_008 score: {total_008:.6f}')
print(f'exp_009 score: {total_009:.6f}')
print(f'Improvement: {total_008 - total_009:.6f}')

# Count improvements per N
improvements = []
for n in range(1, 201):
    diff = scores_008[n] - scores_009[n]
    if diff > 0.0001:
        improvements.append((n, diff))

print(f'\nN values where exp_009 is better: {len(improvements)}')
print(f'Top 10 improvements:')
for n, diff in sorted(improvements, key=lambda x: -x[1])[:10]:
    print(f'  N={n}: +{diff:.6f}')